In [28]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision import models
from PIL import Image
import torch.nn as nn
import torch.optim as optim

In [29]:
class FruitsDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.label_to_index = {}

        # Load image paths and labels
        self._load_data()

    def _load_data(self):
        for idx, label in enumerate(os.listdir(self.root_dir)):
            label_dir = os.path.join(self.root_dir, label)
            if os.path.isdir(label_dir):
                self.label_to_index[label] = idx  # Map label to index
                for image_name in os.listdir(label_dir):
                    self.image_paths.append(os.path.join(label_dir, image_name))
                    self.labels.append(idx)  # Store index instead of label

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label  

In [30]:
data_transforms = transforms.Compose([
    transforms.Resize((100, 100)), 
    transforms.ToTensor(),            
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

In [31]:
dataset = FruitsDataset(root_dir='/home/clindaru/MDST/project_team/mdst-w24/fruits-360_dataset_100x100/fruits-360/Training', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [61]:

# Define a simple CNN model
class custom_cnn(nn.Module):
    def __init__(self):
        super(custom_cnn, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 25 * 25, 128)  # Adjust based on image size
        self.fc2 = nn.Linear(128, len(set(dataset.labels)))  # Adjust based on number of classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 25 * 25)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
model = custom_cnn()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

print('Finished Training')

Epoch [1/10], Loss: 0.2531
Epoch [2/10], Loss: 0.0318
Epoch [3/10], Loss: 0.0190
Epoch [4/10], Loss: 0.0238
Epoch [5/10], Loss: 0.0047
Epoch [6/10], Loss: 0.0161
Epoch [7/10], Loss: 0.0178
Epoch [8/10], Loss: 0.0121
Epoch [9/10], Loss: 0.0114
Epoch [10/10], Loss: 0.0137
Finished Training


In [62]:
torch.save(model.state_dict(), 'fruit_classifier.pth')
print("Model saved as 'fruit_classifier.pth'.")

Model saved as 'fruit_classifier.pth'.


In [70]:
def evaluate_model(model, dataloader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model: {accuracy:.2f}%')


In [72]:
def predict_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension

    # Make prediction
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)

    return predicted.item()  # Return the predicted class index

In [ ]:
test_image_path = '/home/clindaru/MDST/project_team/mdst-w24/fruits-360_dataset_100x100/fruits-360/Training/Apple 6/r1_300_100.jpg'
predicted_label_index = predict_image(test_image_path)
# Map the predicted index back to the label
index_to_label = {v: k for k, v in dataset.label_to_index.items()}
predicted_label = index_to_label[predicted_label_index]

print(f'The predicted label for the image is: {predicted_label}')

fruit_name = predicted_label.split()[0]  

print(f'The predicted label for the image is: {fruit_name}')

The predicted label for the image is: Apple 6
The predicted label for the image is: Apple


In [71]:
val_dataset = FruitsDataset(root_dir='/home/clindaru/MDST/project_team/mdst-w24/fruits-360_dataset_100x100/fruits-360/Test', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Evaluate the model
evaluate_model(model, val_loader)

Accuracy of the model: 93.56%


In [121]:
import requests


def fetch_nutrition_data(fruit_name: str) -> dict:
    app_id = 'Replace with API ID'  # Replace with your actual API ID
    app_key = 'Replace with API Key'  # Replace with your actual API Key
    base_url = 'https://trackapi.nutritionix.com/v2/search/instant/'
    
    # Construct the full URL correctly
    url = f'{base_url}?app_id={app_id}&app_key={app_key}&ingr={fruit_name}'
    print(f"Making request to: {url}")

    try:
        response = requests.get(url)
        response.raise_for_status()  
        return response.json()  
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  
        print(f"Response content: {response.text}") 
    except Exception as err:
        print(f"An error occurred: {err}")

    return None  


nutrition_info = fetch_nutrition_data(fruit_name)


Making request to: https://trackapi.nutritionix.com/v2/search/instant?app_id=304564dd&app_key=9b1e177f6fe51ca7a40c99d97d98395f&ingr=hamburger
HTTP error occurred: 400 Client Error: Bad Request for url: https://trackapi.nutritionix.com/v2/search/instant?app_id=304564dd&app_key=9b1e177f6fe51ca7a40c99d97d98395f&ingr=hamburger
Response content: {"message":"child \"query\" fails because [\"query\" is required]","id":"485141e8-ee44-47c9-bc95-879fcb40eaf5"}
